In [ ]:
%%HTML <style>pre { white-space: pre !important; }</style>

In [ ]:
import org.apache.spark.sql.{SparkSession, DataFrame}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
val contractsDf = spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","true")
        .csv("../../resources/data/csv/contracts.csv")
contractsDf.show(2)

In [ ]:
import org.apache.spark.sql.{functions => f}

#### Transformaciones

In [ ]:
// Windows
    // Agregacion -> max, min, count, avg, mean, sum, ...
    // Ranking -> row_number, rank, dense_rank, ...
    // lag lead

import org.apache.spark.sql.expressions.Window

contractsDf.orderBy(f.col("cod_iuc")).show()

val window1 = Window.partitionBy(f.col("cod_iuc"))
val window2 = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").asc)


contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.count("*").over(window1).alias("count_w") :+
    f.max(f.col("fec_alta")).over(window2).alias("max_fec_alta_w") :+
    f.sum(f.col("cod_producto")).over(window2).alias("sum_cod_producto_w") :_*
).show()

In [ ]:
contractsDf.orderBy(f.col("cod_iuc")).show()

val window3 = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("cod_titular").asc)


contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.row_number().over(window3).alias("row_number") :+
    f.rank().over(window3).alias("rank") :+
    f.dense_rank().over(window3).alias("dense_rank") :_*
).show(100)

In [ ]:
contractsDf.orderBy(f.col("cod_iuc")).show()

val window4 = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").desc)


contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.lag(f.col("fec_alta"), 1, "1970-01-01").over(window4).alias("lag") :+
    f.lead(f.col("fec_alta"), 1, "1970-01-01").over(window4).alias("lead") :_*
).show(100)

In [ ]:
// na.functions
import org.apache.spark.sql.Column

val condColumn: (String, Int) => Column = (colName, threshold) => f.when(f.round(f.rand()*10, 0) >= threshold, null).otherwise(f.col(colName))

val nullContractsDf = contractsDf
    .orderBy("cod_iuc", "cod_titular")
    .select(
        f.col("cod_iuc"), f.col("cod_titular"),
        condColumn("cod_producto", 5).alias("cod_producto"),
        condColumn("fec_alta", 5).alias("fec_alta"),
        condColumn("activo", 5).alias("activo")
    )
nullContractsDf.show()

In [ ]:
// na.functions
    // drop
    // fill
    // replace
nullContractsDf.na
    .drop(
        "all",
        Seq("cod_producto", "fec_alta", "activo")
         )
    .orderBy("cod_iuc", "cod_titular")
    .show()

In [ ]:
nullContractsDf.na
    .fill(
        Map("cod_producto"-> 10000.0, "activo"-> true, "fec_alta" -> "1970-01-01")
        )
    .orderBy("cod_iuc", "cod_titular")
    .show()

In [ ]:
nullContractsDf.na
    .replace(
        Seq("cod_producto", "activo"),
        Map(100 -> 10000, 150 -> 15000)
        )
    .orderBy("cod_iuc", "cod_titular")
    .show()

In [ ]:
// sample
println(contractsDf.count())
contractsDf.sample(true, 0.5, 0).orderBy(f.col("cod_iuc")).show()

In [ ]:
// except / exceptAll
contractsDf.union(contractsDf).exceptAll(contractsDf.filter(f.col("activo") === true).union(contractsDf.filter(f.col("activo") === true))).show(100)